In [ ]:
!pip install mlflow dagshub optuna imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import dagshub
import mlflow
dagshub.init(repo_owner='MihirPrajapati04', repo_name='yt-comment-sentiment-analysis', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/MihirPrajapati04/yt-comment-sentiment-analysis.mlflow")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=42fc4688-7ce1-47ce-96fc-32443fc3e22d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a41f20ac07b423e39c3274a4da53351ca36565974e71d412f859b68e725e7527




Output()

Accessing as MihirPrajapati04

Initialized MLflow to track repo "MihirPrajapati04/yt-comment-sentiment-analysis"

Repository MihirPrajapati04/yt-comment-sentiment-analysis initialized!

In [ ]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/2c18439aef6f4d77be36aad757b9cb04', creation_time=1743247506507, experiment_id='4', last_update_time=1743247506507, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [ ]:
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [ ]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multinomial Naive Bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2025-03-29 11:44:46,267] A new study created in memory with name: no-name-c3028af2-103e-4b54-b466-e091d6165917
[I 2025-03-29 11:44:46,289] Trial 0 finished with value: 0.6684633269921793 and parameters: {'alpha': 0.002001971399356597}. Best is trial 0 with value: 0.6684633269921793.
[I 2025-03-29 11:44:46,309] Trial 1 finished with value: 0.6682519551891778 and parameters: {'alpha': 0.013779738497712149}. Best is trial 0 with value: 0.6684633269921793.
[I 2025-03-29 11:44:46,329] Trial 2 finished with value: 0.6682519551891778 and parameters: {'alpha': 0.0007562470874005807}. Best is trial 0 with value: 0.6684633269921793.
[I 2025-03-29 11:44:46,347] Trial 3 finished with value: 0.6682519551891778 and parameters: {'alpha': 0.013275882232950845}. Best is trial 0 with value: 0.6684633269921793.
[I 2025-03-29 11:44:46,364] Trial 4 finished with value: 0.6682519551891778 and parameters: {'alpha': 0.00010078202987520244}. Best is trial 0 with value: 0.6684633269921793.
[I 2025-03-29 11:4

🏃 View run MultinomialNB_SMOTE_TFIDF_Trigrams at: https://dagshub.com/MihirPrajapati04/yt-comment-sentiment-analysis.mlflow/#/experiments/4/runs/08e11aa2458c43aeb00ebb94ed325982
🧪 View experiment at: https://dagshub.com/MihirPrajapati04/yt-comment-sentiment-analysis.mlflow/#/experiments/4
